# Практика для метрик классификации

Реализуем бинарную классификацию записей голосов (мужских и женских) на основе характеристик записей их разговоров.

В предложенном датасете есть ряд свойств, определенных по аудиозаписям:

- meanfreq: средняя частота голоса (в кГц)
- sd: стандартное отклонение частоты голоса
- median: медианная частота (в кГц)
- Q25: значение в первом квартиле (в кГц)
- Q75: значение в третьем квартиле (в кГц)
- IQR: интерквартильный размах (в кГц)
- skew: ассиметрия
- kurt: эксцесс
- sp.ent: спектральная энтропия
- sfm: энтропия Винера
- mode: мода частоты
- centroid: частотный центроид
- meanfun: средняя основная частота, измеренная по акустическому сигналу
- minfun:  минимальная основная частота, измеренная по акустическому сигналу
- maxfun: максимальная основная частота, измеренная в акустическом сигнале
- meandom: среднее значение доминирующей частоты, измеренной по акустическому сигналу
- mindom: минимум доминирующей частоты, измеренной в акустическом сигнале
- maxdom: максимум доминирующей частоты, измеренной в акустическом сигнале
- dfrange: диапазон доминантных частот, измеренное на звуковой сигнал
- modindx: индекс модуляции голоса

Прежде, чем проводить анализ, можно с помощью визуализации оценить, есть ли различия в частотах голосов мужчин и женщин. Можем построить распределения, например, средних частот:

![voices](./img/classification_practice.png)

Красному цвету соответствуют данные для женских голосов, голубому — для мужских. В целом можно отметить, что распределение частот женских голосов сдвинуто вправо относительно распределения мужских.

Итак, характеристики для голосов различаются, попробуем теперь реализовать классификацию на их основе.


## Задание

Попробуйте построить модель, предсказывающую пол обладателя записи голоса.

Для этого:

Разделите выборку на обучающую и тренировочную с параметрами `test_size=0.3, random_state=42`.

Нормализуйте признаки с помощью функции `StandardScaler()`. Учитывайте, что нормализация тестовой выборки производится по среднему и отклонению тренировочной, которую мы считаем репрезентативной относительно генеральной совокупности.

Обучите модель логистической регрессии на подготовленных данных.

### Пояснение
Тестовые данные не должны влиять на параметры нормализации. Нужно использовать `SCALER.TRANSFORM` вместо `SCALER.FIT_TRANSFORM`, чтобы применять параметры нормализации, рассчитанные для тренировочных данных. Иначе данные в трейне и в тесте будут нормализованы по - разному.

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score 
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np

In [2]:
voice_data = pd.read_csv('./data/voiceDataSet.csv')

In [3]:
voice_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   meanfreq  3168 non-null   float64
 1   sd        3168 non-null   float64
 2   median    3168 non-null   float64
 3   Q25       3168 non-null   float64
 4   Q75       3168 non-null   float64
 5   IQR       3168 non-null   float64
 6   skew      3168 non-null   float64
 7   kurt      3168 non-null   float64
 8   sp.ent    3168 non-null   float64
 9   sfm       3168 non-null   float64
 10  mode      3168 non-null   float64
 11  centroid  3168 non-null   float64
 12  meanfun   3168 non-null   float64
 13  minfun    3168 non-null   float64
 14  maxfun    3168 non-null   float64
 15  meandom   3168 non-null   float64
 16  mindom    3168 non-null   float64
 17  maxdom    3168 non-null   float64
 18  dfrange   3168 non-null   float64
 19  modindx   3168 non-null   float64
 20  label     3168 non-null   obje

In [4]:
voice_data.isna().sum()

meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64

In [5]:
voice_data.sample(5)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
229,0.128033,0.074462,0.112872,0.084791,0.203661,0.118870,3.851646,25.189560,0.935119,0.642840,...,0.128033,0.102382,0.015857,0.228571,0.076250,0.007812,0.140625,0.132812,0.483376,male
1913,0.078847,0.068473,0.054163,0.022843,0.127637,0.104794,2.422417,9.575713,0.930185,0.581113,...,0.078847,0.138373,0.016293,0.271186,0.436663,0.007812,5.328125,5.320312,0.106574,female
345,0.158040,0.060640,0.163175,0.103704,0.206772,0.103069,3.046155,15.197763,0.914717,0.493013,...,0.158040,0.103934,0.034014,0.166667,0.646307,0.092773,3.828125,3.735352,0.258693,male
1580,0.164135,0.060230,0.171909,0.108091,0.219123,0.111032,2.954296,14.333682,0.901312,0.385642,...,0.164135,0.104192,0.013369,0.222222,0.429145,0.004883,0.795898,0.791016,0.554321,male
884,0.194211,0.057748,0.213587,0.136515,0.243104,0.106589,1.594901,5.675729,0.902017,0.306216,...,0.194211,0.127156,0.049180,0.272727,1.241709,0.023438,9.468750,9.445312,0.128863,male


In [6]:
voice_data['label'] = voice_data['label'].apply(lambda x: 1 if x == 'male' else 0)

In [7]:
X = voice_data.drop(columns=['label'])
y = voice_data['label']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [9]:
# scaler = StandardScaler()

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [10]:
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('regression', LogisticRegression())
])

In [11]:
model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)

np.round(accuracy_score(y_test, y_pred), 3)

0.973